In [73]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import recordlinkage
from recordlinkage.datasets import load_febrl2

In [74]:
#the given dataset in the home work
dataset = pd.read_csv(r"D:\datasets\Deduplication Problem - Sample Dataset.csv")
print(dataset.head())

#dataset import from recordLinkage library consisting of 5000 recods with 1000 records being duplicated with max frequency
#of duplicacy 5 entries of a record
dataset1 = load_febrl2()

print(dataset1.head())

             ln       dob gn       fn
0      SMITH JR  01/03/68  F  WILLIAM
1  ROTHMEYER JR  01/03/68  F  WILLIAM
2       ASBY JR  01/03/68  F  WILLIAM
3     SALTER JR  01/03/68  F  WILLIAM
4     SALTER JR  01/03/68  F  WILLIAM
              given_name    surname street_number         address_1  \
rec_id                                                                
rec-2778-org       sarah      bruhn            44     forbes street   
rec-712-dup-0      jacob     lanyon             5        milne cove   
rec-1321-org     brinley  efthimiou            35  sturdee crescent   
rec-3004-org     aleisha     hobson            54     oliver street   
rec-1384-org       ethan    gazzola            49    sheaffe street   

                address_2              suburb postcode state date_of_birth  \
rec_id                                                                       
rec-2778-org   wintersloe        kellerberrin     4510   vic      19300213   
rec-712-dup-0     wellwod  beaconsfield 

In [75]:
#drop useless columns for data cleaning and its important feature extraction
dataset1.drop(['soc_sec_id' , 'address_2' , 'suburb' , 'postcode' , 'street_number' , 'address_1' , 'state' ] , inplace  = True  , axis = 1)
print(dataset1.head())
dataset.drop(['gn' ] , inplace  = True  , axis = 1)
print(dataset.head())


              given_name    surname date_of_birth
rec_id                                           
rec-2778-org       sarah      bruhn      19300213
rec-712-dup-0      jacob     lanyon      19080712
rec-1321-org     brinley  efthimiou      19940319
rec-3004-org     aleisha     hobson      19290427
rec-1384-org       ethan    gazzola      19631225
             ln       dob       fn
0      SMITH JR  01/03/68  WILLIAM
1  ROTHMEYER JR  01/03/68  WILLIAM
2       ASBY JR  01/03/68  WILLIAM
3     SALTER JR  01/03/68  WILLIAM
4     SALTER JR  01/03/68  WILLIAM


In [76]:
dataset2 = pd.DataFrame()
dataset2['ln'] = dataset1['surname']
dataset2['dob'] = dataset1['date_of_birth']
dataset2['fn'] = dataset1['given_name']
print(dataset2.head())

                      ln       dob       fn
rec_id                                     
rec-2778-org       bruhn  19300213    sarah
rec-712-dup-0     lanyon  19080712    jacob
rec-1321-org   efthimiou  19940319  brinley
rec-3004-org      hobson  19290427  aleisha
rec-1384-org     gazzola  19631225    ethan


In [77]:
def clean_name(name):
    sl = name.split(" ")
    
    if sl[0] == 'Mr.' or sl[0] == 'Mrs.' or sl[0] == 'Sir' or sl[0] == 'Mam' or sl[0] == 'Sr.' or sl[0] == 'Jr.' :
        return s1[1]
    #the name is returned after designation
    else:
        return sl[0]
    #if no respectable designation is found the return name


def format_dob(dob):
    dob = dob.replace("/" , "")
    #format dob by removing '/' to make it just a number value , simple to work in numpy
    return dob

In [78]:
#data preprocessing and cleaning step 
dataset['ln'] = dataset['ln'].apply(clean_name)
dataset['fn'] = dataset['fn'].apply(clean_name)
dataset['dob'] = dataset['dob'].apply(format_dob)
print(dataset.head())

          ln     dob       fn
0      SMITH  010368  WILLIAM
1  ROTHMEYER  010368  WILLIAM
2       ASBY  010368  WILLIAM
3     SALTER  010368  WILLIAM
4     SALTER  010368  WILLIAM


In [79]:
#merging two datasets together and replacing 'NaN' values with a particular dummy value

dataset = pd.concat([dataset2 , dataset])
dataset = dataset.fillna(value = {'fn' : 'fernandez' , 'dob' : '19300311'})

print(dataset)


                        ln       dob         fn
rec-2778-org         bruhn  19300213      sarah
rec-712-dup-0       lanyon  19080712      jacob
rec-1321-org     efthimiou  19940319    brinley
rec-3004-org        hobson  19290427    aleisha
rec-1384-org       gazzola  19631225      ethan
rec-3981-org          hope  19421201     alicia
rec-916-org       kolosche  19450918   benjamin
rec-1684-org        lusted  19950620   petreece
rec-63-dup-0         white  19000106     olivia
rec-3808-org       maynard  19150402  fernandez
rec-112-org           rudd  19951125     joshua
rec-3297-org        lomman  19910228    rachael
rec-1315-org     dissinger  19370312     joseph
rec-1050-org         verco  19400826      sarah
rec-2116-org       burford  19471114    sidonie
rec-3232-org       carbone  19720809     andrew
rec-1900-dup-1     everett  19300311      kiara
rec-2460-dup-2    hathaway  19390305   nicholas
rec-3123-org        bastin  19580628   isabella
rec-2166-org      matthews  19261017  al

In [80]:
dataset = np.array(dataset)
dataset = pd.DataFrame(dataset)
# above two lines are used to make index values same
data = pd.DataFrame()
data['ln'] = dataset[0]
data['dob'] = dataset[1]
data['fn'] = dataset[2]

print(data.head())

#the similar feilds are merged together through block Indexing where the most similar objects are clubbed together
#thus helps in minimizing the number of comparisions between various values
indexer = recordlinkage.BlockIndex(on= ['fn' , 'ln']) 
pairs = indexer.index(data)

print (len(pairs))
#prints the pairs using blocking method which is used to pair similar data feilds together

          ln       dob       fn
0      bruhn  19300213    sarah
1     lanyon  19080712    jacob
2  efthimiou  19940319  brinley
3     hobson  19290427  aleisha
4    gazzola  19631225    ethan
1143


In [81]:
compare_cl = recordlinkage.Compare()
#this is used to compare the values with each other for clubing the duplicacies together
compare_cl.exact('fn', 'fn', label='fn')
compare_cl.string('ln', 'ln', method='jarowinkler', threshold=0.85, label='n')
compare_cl.exact('dob', 'dob', label='dob')

features = compare_cl.compute(pairs, data)
#compares all possible pairs formed with the dataset to the dataset

features.head(10)


fn    n  dob
1   4332   1  1.0    1
7   3055   1  1.0    1
8   90     1  1.0    1
    2827   1  1.0    1
90  2827   1  1.0    1
9   2463   1  1.0    0
10  1966   1  1.0    1
16  390    1  1.0    1
    1239   1  1.0    1
390 1239   1  1.0    1

In [82]:
features.describe()
#prints the box plot values ie. 5-value summary 

,fn,n,dob
count,1143.0,1143.0,1143.000000
mean,1.0,1.0,0.647419
std,0.0,0.0,0.477983
min,1.0,1.0,0.000000
25%,1.0,1.0,0.000000
50%,1.0,1.0,1.000000
75%,1.0,1.0,1.000000
max,1.0,1.0,1.000000


In [83]:
features.sum(axis=1).value_counts().sort_index(ascending=False)
#prints those no. of entries with no. of columns matched

3.0    740
2.0    403
dtype: int64

In [87]:
#print(features.sum(axis=1))
matches = features[features.sum(axis=1) > 2]
#features.sum(axis=1) > 2 , means those data feilds with all the three entries same ie. the duplicated values show up here..
print('the unique entries : ' ,len(matches))
#matches prints the duplicated values in the data and thus return all the duplicated names related to a person
print(matches.head(10))
print("the no. of duplicated entries are " , len(data) - len(matches))

the unique entries :  740
          fn    n  dob
1   4332   1  1.0    1
7   3055   1  1.0    1
8   90     1  1.0    1
    2827   1  1.0    1
90  2827   1  1.0    1
10  1966   1  1.0    1
16  390    1  1.0    1
    1239   1  1.0    1
390 1239   1  1.0    1
17  3099   1  1.0    1
the no. of duplicated entries are  4363
